## How to Train YOLOv8 Object Detection on a Custom Dataset

In [ ]:
!gdown 1u6uANPoai_N6ytg9zFbT2ISYVUqWtJ_X

Downloading...
From: https://drive.google.com/uc?id=1u6uANPoai_N6ytg9zFbT2ISYVUqWtJ_X
To: /content/train_dataset_Минприроды.zip
100% 1.76G/1.76G [00:11<00:00, 148MB/s]


In [ ]:
!pip install hugsvision
!pip install --upgrade accelerate
!pip install accelerate -U

In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.28.0

In [7]:
!pip install rasterio
!pip install pybboxes
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 62.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.5/586.5 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 24.5 MB/s eta 0:00:00


In [ ]:
!unzip /content/train_dataset_Минприроды.zip

In [ ]:
!unzip /content/train_dataset_Минприроды/разметка_кликун.zip
!unzip /content/train_dataset_Минприроды/разметка_малый.zip
!unzip /content/train_dataset_Минприроды/разметка_шипун.zip

In [ ]:
!rm -rf /content/разметка_шипун/images/masks

In [ ]:
import numpy as np
import pybboxes as pbx
from PIL import Image
from skimage import draw
import random
from pathlib import Path
import os
import shutil

from rasterio import features
from google.colab.patches import cv2_imshow
import cv2
import numpy as np

In [ ]:
#!rm -rf /content/FORMODEL

In [ ]:
os.makedirs('/content/FORMODEL/')
os.makedirs('/content/FORMODEL/train/')
os.makedirs('/content/FORMODEL/val/')
os.makedirs('/content/FORMODEL/train/images')
os.makedirs('/content/FORMODEL/train/labels')
os.makedirs('/content/FORMODEL/val/images')
os.makedirs('/content/FORMODEL/val/labels')

In [ ]:
def resize_image_and_label(image, label):
    height, width, channels = image.shape
    label_height, label_width = label.shape
    
    square_size = max(height, width)
    
    square_image = np.zeros((square_size, square_size, channels), dtype=np.uint8)
    x_offset = (square_size - width) // 2
    y_offset = (square_size - height) // 2
    square_image[y_offset:y_offset+height, x_offset:x_offset+width] = image

    square_label = np.zeros((square_size, square_size), dtype=np.uint8)
    square_label[y_offset:y_offset+height, x_offset:x_offset+width] = label
    
    line_pixels = (square_image.sum(axis=2) == 0)  # Pixels where all channels are 0 (black lines)
    noise = np.random.randint(200, 256, (square_size, square_size, channels), dtype=np.uint8)
    sky_image = np.where(line_pixels[..., np.newaxis], noise, square_image)

    resized_image = cv2.resize(sky_image, (640, 640))
    resized_image = cv2.normalize(resized_image, resized_image, 0, 255, cv2.NORM_MINMAX)

    resized_label = cv2.resize(square_label, (640, 640), interpolation=cv2.INTER_NEAREST)
    
    return resized_image, resized_label

In [ ]:
def seg_to_bbox(label, label_class):
    #image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)

    unique_gray_values = np.unique(label)
    unique_gray_values = unique_gray_values[unique_gray_values != 0]

    coordinates = []

    for gray_value in unique_gray_values:
        indices = np.where(label == gray_value)
        
        min_x = np.min(indices[1])
        max_x = np.max(indices[1])
        min_y = np.min(indices[0])
        max_y = np.max(indices[0])
        
        coordinates.append((min_x, min_y, max_x, max_y))

    return coordinates

In [ ]:
def visualize_yolo_bounding_boxes(image, bounding_boxes):
    for bbox in bounding_boxes:
        class_id, x_center, y_center, width, height = bbox.split()
        
        class_id = int(class_id)
        x_center = float(x_center)
        y_center = float(y_center)
        width = float(width)
        height = float(height)
        
        x_min = int((x_center - width / 2) * image.shape[1])
        y_min = int((y_center - height / 2) * image.shape[0])
        x_max = int((x_center + width / 2) * image.shape[1])
        y_max = int((y_center + height / 2) * image.shape[0])
        
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        label = f"Class {class_id}"
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2_imshow(image)

In [ ]:
def prepare_images(class_label, image_name, image_path, label_path, load_path):
    image = cv2.imread(image_path+image_name+'.jpg')
    label = cv2.imread(label_path+image_name+'.png', 0)

    try:
        resized_image, resized_label = resize_image_and_label(image, label)
    except Exception as e:
        print(e)
        return
    
    #cv2_imshow(resized_image)

    cv2.imwrite(os.path.join(load_path + '/images/', image_name+'.jpg'), resized_image)

    boxes = seg_to_bbox(resized_label, 0)
    image_height, image_width = resized_label.shape

    W, H = image_width, image_height

    ans = []
    for box in boxes:
        a1,a2,a3,a4 = pbx.convert_bbox(box, from_type="voc", to_type="yolo", image_size=(W,H))
        ans.append(f"{class_label} {a1} {a2} {a3} {a4}\n")
    
    #cv2_imshow(visualize_yolo_bounding_boxes(resized_image, ans))

    with open(load_path+'labels/'+image_name+'.txt', "w") as file:
        file.write("\n".join(ans))

In [ ]:
#prepare_images(0, '1211999571', '/content/klikun/images/', '/content/klikun/masks/', '/content/FORMODEL/train/')

In [ ]:
#len(os.listdir('/content/ship/images'))

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
for k, filepath in tqdm(enumerate(os.listdir('/content/klikun/images')[:])):
    if k>1100:break

    if k<1000:
        load_path = '/content/FORMODEL/train/'
    else:
        load_path = '/content/FORMODEL/val/'
    
    prepare_images(0, filepath.replace('.jpg', ''), '/content/klikun/images/', '/content/klikun/masks/', load_path)

0it [00:00, ?it/s]

could not broadcast input array from shape (2048,1152) into shape (1009,2048)


In [ ]:
for k, filepath in tqdm(enumerate(os.listdir('/content/mal/images')[:])):
    if k>1100:break

    if k<1000:
        load_path = '/content/FORMODEL/train/'
    else:
        load_path = '/content/FORMODEL/val/'
    
    prepare_images(1, filepath.replace('.jpg', ''), '/content/mal/images/', '/content/mal/masks/', load_path)

0it [00:00, ?it/s]

In [ ]:
for k, filepath in tqdm(enumerate(os.listdir('/content/ship/images')[:])):
    if k>1100:break

    if k<1000:
        load_path = '/content/FORMODEL/train/'
    else:
        load_path = '/content/FORMODEL/val/'
    
    prepare_images(2, filepath.replace('.jpg', ''), '/content/ship/images/', '/content/ship/masks/', load_path)

0it [00:00, ?it/s]

In [ ]:
yaml_content = f'''
path: /content/FORMODEL/
train: /content/FORMODEL/train
val: /content/FORMODEL/val
test: /content/FORMODEL/val

names:
  0: 'mal'
  1: 'klikun'
  2: 'ship'
'''
#['klikun', 'mal', 'ship']
with Path('data.yaml').open('w') as f:
    f.write(yaml_content)

In [ ]:
#from google.colab import files
#files.download('/content/runs/detect/train2/weights/best.pt')

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
model.to('cuda:0')

In [ ]:
results = model.train(
        batch=32,
        device="cuda:0",
        data="data.yaml",
        epochs=10,
        imgsz=640,
    )

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/content/runs/detect/train2/val_batch0_pred.jpg")

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!zip -r /content/FORMODEL.zip /content/FORMODEL 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/FORMODEL.zip' '/content/drive/MyDrive/FORMODEL.zip'

In [ ]:
def visualize_yolo_res(image, bounding_boxes):
    for bbox in bounding_boxes:
        print(bbox)
        x_center, y_center, width, height, clas = bbox.split()
        
        # Convert string values to float
        class_id = int(class_id)
        x_center = float(x_center)
        y_center = float(y_center)
        width = float(width)
        height = float(height)
        
        # Calculate the coordinates of the bounding box
        x_min = int((x_center - width / 2) * image.shape[1])
        y_min = int((y_center - height / 2) * image.shape[0])
        x_max = int((x_center + width / 2) * image.shape[1])
        y_max = int((y_center + height / 2) * image.shape[0])
        
        # Draw the bounding box rectangle on the image
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        # Display the class label on the bounding box
        label = f"Class {class_id}"
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Show the image with bounding boxes
    return image

# **PREDICTIONS**

In [ ]:
!gdown 1OeyFFgmLHSkZ2uWRZIdqad0_IAtdI14c

In [ ]:
!unzip /content/HUGSMODEL.zip

In [ ]:
#model bin
#!gdown 1kr9EkvRYRqA7B2C_UzO-jdC99FhgwZcl

In [ ]:
!gdown 1WZxaj3GBd8xkZJ_FLV_7Mv3dQaISE3p5

In [ ]:
#import torch
#state_dict = torch.load('/content/hugs/pytorch_model.bin', map_location="cpu")

In [ ]:
from ultralytics import YOLO
from PIL import Image
from numpy import asarray

import os.path
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

yolo_model = YOLO('/content/best.pt')

hugs_path = "/content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer"
hugs_model = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained(hugs_path),
    model = ViTForImageClassification.from_pretrained(hugs_path),
)

classes = {
    'mal': 0,
    'skikun': 1,
    'ship': 2
}

In [57]:
def gen_ansambel(path):
    ####################
    # YOLO PREDICTION DOWN
    ###################
    image = Image.open(path)
    image = image.convert('RGB')
    image = asarray(image)

    res = yolo_model.predict(image)

    class_c = {
        0: 0.,
        1: 0.,
        2: 0.
    }

    for i, clas in enumerate(res[0].boxes.cls):
        clas = int(clas)

        class_c[clas] += float(res[0].boxes.conf[i].item())
    
    yolo_class = max(class_c, key=class_c.get)
    # average conf
    yolo_conf = class_c[yolo_class] / len(res[0].boxes.cls)
    ###################
    # YOLO PRED UP
    ###################

    ###################
    # HUGS PRED DOWN
    ###################
    try:
        hugs_class = hugs_model.predict(img_path=path)
        hugs_class = classes[hugs_class]
    except Exception as e:
        hugs_class = -1
        print('HUGS FAILED ON ' + path)
    ################### + path
    # HUGS PRED UP
    ###################
    
    final_pred = 0
    if hugs_class == -1:
        final_pred = yolo_class
    else:
        if yolo_class != hugs_class:
            if yolo_conf >= 0.5:
                final_pred = yolo_class
            else:
                final_pred = hugs_class
        else:
            final_pred = yolo_class
    
    print(f'Yolo class {yolo_class}, yolo conf {yolo_conf}, Hugs class {hugs_class}')

    image = cv2.imread(path)

    boxes_list = np.array(res[0].boxes.boxes.cpu()).tolist()

    for _ in boxes_list:
        xmin, ymin, xmax, ymax, pro, cl = _[0], _[1], _[2], _[3], _[4], _[5]
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
    
    return final_pred+1, image

In [58]:
def gen_yolo(path):
    ####################
    # YOLO PREDICTION DOWN
    ###################
    image = Image.open(path)
    image = image.convert('RGB')
    image = asarray(image)

    res = yolo_model.predict(image)

    class_c = {
        0: 0.,
        1: 0.,
        2: 0.
    }

    for i, clas in enumerate(res[0].boxes.cls):
        clas = int(clas)

        class_c[clas] += float(res[0].boxes.conf[i].item())
    
    yolo_class = max(class_c, key=class_c.get)
    # average conf
    yolo_conf = class_c[yolo_class] / len(res[0].boxes.cls)
    ###################
    # YOLO PRED UP
    ###################

    ###################
    # HUGS PRED DOWN
    ###################
    #hugs_class = hugs_model.predict(img_path=path)
    #hugs_class = classes[hugs_class]
    ###################
    # HUGS PRED UP
    ###################
    
    final_pred = yolo_class
    #if yolo_class != hugs_class:
    #    if yolo_conf >= 0.5:
    #        final_pred = yolo_class
    #    else:
    #        final_pred = hugs_class
    #else:
    #    final_pred = yolo_class
    
    #print(f'Yolo class {yolo_class}, yolo conf {yolo_conf}, Hugs class {hugs_class}')

    image = cv2.imread(path)

    boxes_list = np.array(res[0].boxes.boxes.cpu()).tolist()

    for _ in boxes_list:
        xmin, ymin, xmax, ymax, pro, cl = _[0], _[1], _[2], _[3], _[4], _[5]
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
    
    return final_pred+1, image

In [38]:
gen('/content/1680723390_animals-pibig-info-p-staya-lebedei-zhivotnie-krasivo-1.jpg')


0: 448x640 8 ships, 27.9ms
Speed: 2.8ms preprocess, 27.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


Yolo class 2, yolo conf 0.531843576580286, Hugs class 2


3

In [ ]:
'''image = cv2.imread(path)

    boxes_list = np.array(res[0].boxes.boxes.cpu()).tolist()

    for _ in boxes_list:
        xmin, ymin, xmax, ymax, pro, cl = _[0], _[1], _[2], _[3], _[4], _[5]
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
    
    #boxed_img = visualize_yolo_res(image, np.array(res[0].boxes.boxes.cpu()).tolist())

    #cv2_imshow(image)

    #print( "Количество особей: "+str((res[0].boxes.cls)) )
    #print(res[0].boxes.boxes)'''

In [43]:
!gdown 1_SWQjEWBJOBQzupfI_2NY9Lib-0hZk5U

Downloading...
From: https://drive.google.com/uc?id=1_SWQjEWBJOBQzupfI_2NY9Lib-0hZk5U
To: /content/Тестовый датасет.zip
100% 104M/104M [00:03<00:00, 26.6MB/s] 


In [ ]:
!unzip /content/testdadatset.zip

In [59]:
import pandas as pd
#testd = pd.read_csv('')

SUBM = pd.DataFrame(columns=['name', 'class'])

In [55]:
from tqdm.notebook import tqdm_notebook as tqdm
import cv2
import numpy as np

In [60]:
testfolder = '/content/TESTdataset/'

for filepath in tqdm(os.listdir(testfolder)):
    print(testfolder + filepath)
    #filepath = row['name']

    clas, img = gen_ansambel(testfolder + filepath)
    SUBM.append({'name': filepath, 'class': clas}, ignore_index=True)

  0%|          | 0/595 [00:00<?, ?it/s]


0: 640x640 1 ship, 38.1ms
Speed: 4.8ms preprocess, 38.1ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)


/content/TESTdataset/133.jpg


WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
<ipython-input-60-051a0d118cd5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SUBM.append({'name': filepath, 'class': clas}, ignore_index=True)

0: 480x640 1 klikun, 2 ships, 29.2ms
Speed: 3.3ms preprocess, 29.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


Yolo class 2, yolo conf 0.8972364664077759, Hugs class 2
/content/TESTdataset/59.jpg


...

0: 544x640 1 mal, 36.2ms
Speed: 3.5ms preprocess, 36.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
<ipython-input-60-051a0d118cd5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SUBM.append({'name': filepath, 'class': clas}, ignore_index=True)

0: 512x640 1 klikun, 1 mal, 30.0ms
Speed: 3.4ms preprocess, 30.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


HUGS FAILED ON /content/TESTdataset/502.jpg
Yolo class 1, yolo conf 0.7361446022987366, Hugs class -1
/content/TESTdataset/137.jpg


WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


Yolo class 1, yolo conf 0.22828060388565063, Hugs class 0


<ipython-input-60-051a0d118cd5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SUBM.append({'name': filepath, 'class': clas}, ignore_index=True)


In [61]:
value = SUBM.loc[SUBM["name"] == "510.jpg", "class"].item()

print(value)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/base.py:347 in item                          │
│                                                                                                  │
│    344 │   │   """                                                                               │
│    345 │   │   if len(self) == 1:                                                                │
│    346 │   │   │   return next(iter(self))                                                       │
│ ❱  347 │   │   raise ValueError("can only convert an array of size 1 to a Python scalar")        │
│    348 │                                                                                         │
│    349 │   @property                                                                             │
│    350 │   def nbytes(self) -> int:                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
SUBM.to_csv('/content/SUBM.csv', index=False)

In [39]:
!zip -r /content/HugsModelWeights.zip /content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer

  adding: content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/ (stored 0%)
  adding: content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/config.json (deflated 48%)
  adding: content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/training_args.bin (deflated 48%)
  adding: content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/pytorch_model.bin (deflated 7%)
  adding: content/content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/preprocessor_config.json (deflated 46%)


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
!cp /content/HugsModelWeights.zip /content/drive/MyDrive/